In [4]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

print("="*60)
print("ТЕСТОВЫЙ ЗАПУСК КЛАССИФИКАТОРА")
print("="*60)

# Определяем RuleBasedPreprocessor
class RuleBasedPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, rules_path='../data/rules/rules.json'):
        with open(rules_path, 'r', encoding='utf-8') as f:
            self.rules = json.load(f)
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        def apply_rules(text):
            tags = []
            for cat, keywords in self.rules.items():
                for kw in keywords:
                    if kw.lower() in text.lower():
                        tags.append(cat)
            return text + ' ' + ' '.join(tags)
        return [apply_rules(doc) for doc in X]

# Класс классификатора
class LightweightClassifier:
    def __init__(self):
        self.pipeline = Pipeline([
            ('rules', RuleBasedPreprocessor()),
            ('tfidf', TfidfVectorizer(max_features=500, ngram_range=(1,2))),
            ('clf', MultinomialNB(alpha=0.1))
        ])
    def train(self, texts, labels):
        self.pipeline.fit(texts, labels)
    def predict(self, texts):
        preds = self.pipeline.predict(texts)
        probs = self.pipeline.predict_proba(texts)
        confidences = probs.max(axis=1)
        return list(zip(preds, confidences))
    def save(self, path='../models/lightweight_clf.pkl'):
        import joblib
        joblib.dump(self.pipeline, path)
    def load(self, path='../models/lightweight_clf.pkl'):
        import joblib
        self.pipeline = joblib.load(path)

print("="*60)
print("НАЧАЛО ОБУЧЕНИЯ МОДЕЛИ")
print("="*60)

# Загрузка датасета
df = pd.read_csv('../data/processed/training_data.csv')
print(f"Загружено {len(df)} примеров, категорий: {df['category'].nunique()}")

print("\nОбработка текста...")

# Разделение
X_train, X_test, y_train, y_test = train_test_split(df['text_clean'], df['category'], test_size=0.2, random_state=42)
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

print("\nПостроение TF-IDF признаков...")
clf = LightweightClassifier()
clf.train(X_train, y_train)
print(f"Размерность признакового пространства: {len(clf.pipeline.named_steps['tfidf'].get_feature_names_out())}")

print("\nРЕЗУЛЬТАТЫ:")
# Прогноз
y_pred_conf = clf.predict(X_test.tolist())
y_pred = [p for p, c in y_pred_conf]

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.2%}")

print("\nОтчёт по метрикам:")
print(classification_report(y_test, y_pred, zero_division=0))

print("Матрица ошибок:")
cm = confusion_matrix(y_test, y_pred, labels=sorted(df['category'].unique()))
labels = sorted(df['category'].unique())
import pandas as pd
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print(cm_df)

# Сохраняем модель
clf.save('../models/classifier.pkl')
print("\nМодель сохранена в models/classifier.pkl")

# Пример работы модели
examples = [
    "Docker контейнер не запускается",
    "Не могу подключиться к VPN",
    "Отчет в Power BI не открывается",
    "SAP зависает при открытии транзакции"
]

print("\nПроверка предсказания:\n")
for ex in examples:
    pred, conf = clf.predict([ex])[0]
    print(f"Запрос: {ex}")
    print(f"Категория: {pred} ({conf*100:.2f}%)")
    
    # Выдать топ-3 с вероятностями
    feature_names = clf.pipeline.named_steps['tfidf'].get_feature_names_out()
    clf_step = clf.pipeline.named_steps['clf']
    probas = clf_step.predict_proba(clf.pipeline.named_steps['tfidf'].transform(clf.pipeline.named_steps['rules'].transform([ex])))
    top3_indices = np.argsort(probas[0])[::-1][:3]
    top3 = [(clf_step.classes_[i], probas[0][i]) for i in top3_indices]
    
    print("Топ-3:", top3)
    print()

print("="*60)
print("КОНЕЦ ТЕСТОВОГО ЗАПУСКА")
print("="*60)


ТЕСТОВЫЙ ЗАПУСК КЛАССИФИКАТОРА
НАЧАЛО ОБУЧЕНИЯ МОДЕЛИ
Загружено 1000 примеров, категорий: 9

Обработка текста...
Train: 800, Test: 200

Построение TF-IDF признаков...
Размерность признакового пространства: 500

РЕЗУЛЬТАТЫ:
Точность модели: 81.00%

Отчёт по метрикам:
               precision    recall  f1-score   support

       access       0.00      0.00      0.00         5
    documents       0.92      0.85      0.88        13
       errors       0.68      0.78      0.73        36
miscellaneous       0.00      0.00      0.00         1
        other       0.77      0.67      0.71        15
     requests       0.85      0.95      0.90       109
     software       0.50      0.33      0.40         6
       system       1.00      0.45      0.62        11
  time_period       1.00      0.50      0.67         4

     accuracy                           0.81       200
    macro avg       0.63      0.50      0.55       200
 weighted avg       0.79      0.81      0.79       200

Матрица ошибок: